In [1]:
import os 

import torch 
import torch.nn 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import pickle

from transformers import WhisperProcessor, WhisperTokenizer, WhisperForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import nlptutti as metrics
import torch.optim as optim
from tqdm.auto import tqdm 

import whisper
#from my_model import Mymodel
from data_utils import MinyoDataset, custom_collate_fn, get_wer
from my_model_daewoung import Mymodel




In [2]:
audio_path = '/home/daewoong/userdata/danbi/final_tts_audio'
lyric_path = '/home/daewoong/userdata/danbi/final_lyrics_data/'
each_lyric = '/home/daewoong/userdata/danbi/each_song_lyrics.txt'
result_path = '/home/daewoong/userdata/danbi/encoder_result'
filtered_id_list = pickle.load(open('/home/daewoong/userdata/danbi/thirty_second_filtered_id.pkl', 'rb'))

print('download token now')

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="ko", task="transcribe", predict_timestamps=True)
dataset = MinyoDataset(result_path, lyric_path, processor, filtered_id_list, each_lyric, max_len = 1024)

print('token download complete')

train_size = int(len(dataset) * 0.8)
valid_size = len(dataset) - train_size

train_data, valid_data = random_split(dataset, [train_size, valid_size])

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=custom_collate_fn, num_workers=8, pin_memory=True)
valid_dataloader = DataLoader(valid_data, batch_size=2, shuffle=False, collate_fn=custom_collate_fn, num_workers=8, pin_memory=True)

#next(iter(train_dataloader))


print('load model now')
#pre_model = whisper.load_model("large-v2")
pre_model = whisper.load_model("/home/daewoong/userdata/danbi/whisper_pretrain/large-v2.pt", device='cpu')  
#WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
print('load model complete')

processor.tokenizer.set_prefix_tokens(language="ko", task="transcribe", predict_timestamps=True)

download token now
token download complete
load model now
load model complete


In [3]:

device = 'cuda'
epoch = 4
model_dims = pre_model.dims
model_dims.n_ref_encoder_layer = 2
model_dims.n_ref_decoder_layer = 2
model_dims.n_ref_text_ctx = 1024
model_dims.n_ref_text_state = 240

model = Mymodel(model_dims)

# model.encoder.load_state_dict(pre_model.encoder.state_dict())
# model.decoder.token_embedding.load_state_dict(pre_model.decoder.token_embedding.state_dict())
# model.decoder.blocks.load_state_dict(pre_model.decoder.blocks.state_dict())
# model.decoder.positional_embedding.data=pre_model.decoder.positional_embedding.data.clone()
# model.ref_encoder.token_embedding.load_state_dict(pre_model.decoder.token_embedding.state_dict())


In [48]:

model.load_state_dict(torch.load('/home/daewoong/userdata/danbi/epoch_3.pt', map_location='cpu'))


<All keys matched successfully>

In [49]:
model = model.to(device)

for param in model.encoder.parameters():
  param.requires_grad = False
for param in model.decoder.blocks.parameters():
  param.requires_grad = False

In [50]:
batch = next(iter(valid_dataloader))
audio, input_text, input_txt_attn, around_text, around_txt_attn = batch


In [7]:
audio[0].shape == torch.Size([1500, 1280])

True

In [8]:
# pre_model.eval()
# with torch.no_grad():
#   k = pre_model.transcribe('/home/daewoong/userdata/danbi/final_tts_audio/lyric0-2.wav')  
  
# print(k)

In [85]:
pp = processor.tokenizer('이여도 한양 라 이여도하 한양\n 저녁이나 밝으때 하저\n 이여방애 이여방애\n 이여도 한양 라 이여도 한양 라\n 이여이여 이여도 한양 라\n 한 마슬에 세첩한 놈은, 대천바당 한 가운데, 이여동 한양 라 이여동 한양 라, 집세거리 이신놈은, 살앙간이나 잇건마는, 이여이여 이여도 한양 라, 사랑 사랑은 내 사랑이여, 이여도 한양 라 이여도 한양 라, 간다드니 웨 왓더냐, 돌아사면 후훼 할 질을, 돌아 사지를 말 아니해여 말 아니 해여, 한 해에랑 옷 세벌 입엉, 보리 비단 능라비로, 즉을때 까장 사랑한다고 해여두고, 한강 바당 한가운 듸랑, 치엇은 배질이러라', return_tensors='pt')
token_text, token_text_attn = dataset.around_pad_seqence(pp)

In [105]:
kk = model.transcribe('/home/daewoong/userdata/danbi/final_tts_audio/lyric0-2.wav', around_text=token_text.unsqueeze(0).to(device), temperature=0.0)

torch.Size([1, 1]) torch.Size([1, 1500, 1280]) torch.int64 torch.float16 torch.Size([1, 1024]) torch.int64


In [106]:
kk

{'text': ' 이여이여 이여도 한양 라 한 마슬에 새첩한 놈은',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 11.41,
   'text': ' 이여이여 이여도 한양 라 한 마슬에 새첩한 놈은',
   'tokens': [50364,
    4329,
    245,
    105,
    21064,
    105,
    4329,
    245,
    1698,
    1838,
    4815,
    14973,
    22339,
    4815,
    6437,
    44532,
    1517,
    31184,
    4758,
    102,
    3049,
    10091,
    230,
    2124],
   'temperature': 0.0,
   'avg_logprob': -0.2246149444580078,
   'compression_ratio': 1.032258064516129,
   'no_speech_prob': 1.1295273161238129e-08}],
 'language': 'ko'}

In [89]:
text = ['불매 불매 불매야', '이기 어디메 불매고', '경상도 대불매', '전라도의 자롱쇠', '푸르락 딱닥 불매야', '너이 불머는 대불매', '둘이 불머는 소불매', '혼자 불머는 핸수간', '베렸다 하먼 호미요', '뚜들겼다 하먼 낫이요', '담궜다 하먼 괭이다']
for i in text:
  k = '\n '.join(text)
k = k + '\n'

In [90]:
k

'불매 불매 불매야\n 이기 어디메 불매고\n 경상도 대불매\n 전라도의 자롱쇠\n 푸르락 딱닥 불매야\n 너이 불머는 대불매\n 둘이 불머는 소불매\n 혼자 불머는 핸수간\n 베렸다 하먼 호미요\n 뚜들겼다 하먼 낫이요\n 담궜다 하먼 괭이다\n'

In [91]:
pp = processor.tokenizer(k, return_tensors='pt')
token_text, token_text_attn = dataset.around_pad_seqence(pp)

In [92]:
processor.decode(token_text)

'<|startoftranscript|><|ko|><|transcribe|>불매 불매 불매야\n 이기 어디메 불매고\n 경상도 대불매\n 전라도의 자롱쇠\n 푸르락 딱닥 불매야\n 너이 불머는 대불매\n 둘이 불머는 소불매\n 혼자 불머는 핸수간\n 베렸다 하먼 호미요\n 뚜들겼다 하먼 낫이요\n 담궜다 하먼 괭이다\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

In [127]:
token_text

tensor([50258, 50264, 50359,  ..., 50257, 50257, 50257])

In [173]:
start = torch.tensor([50258])

In [174]:
start = start.unsqueeze(0)

In [175]:
y, sr = torchaudio.load('/home/daewoong/userdata/test_set/test_audio44.wav')
a = processor.feature_extractor(y.mean(0), sampling_rate=sr, return_tensors='pt')
audio = a['input_features']

In [176]:
audio = audio[0]

In [177]:
audio = audio.unsqueeze(0)

In [178]:
audio.shape

torch.Size([1, 80, 3000])

In [200]:
#이여이여 이여도 한양 라 한 마슬에 세첩한 놈은
#'불매 불매 불매야\n 이기 어디메 불매고\n 경상도 대불매\n 전라도의 자롱쇠\n 푸르락 딱닥 불매야\n 너이 불머는 대불매\n 둘이 불머는 소불매\n 혼자 불머는 핸수간\n 베렸다 하먼 호미요\n 뚜들겼다 하먼 낫이요\n 담궜다 하먼 괭이다\n'
#{'text': ' 불메 불메 불메야 이 기어등에 불메고 경상도 대불메 전라도야 자롱새 불락딱딱 불메야 너이 불모는 대불메 둘이 불모는 서불메 혼자 불모는 펜스건 제로따 하면 혼미요 두들겄다 하면 나시요 탕고따 하면 꽤이다 불메야',

model.eval()
with torch.inference_mode():
  kk = model.transcribe('/home/daewoong/userdata/test_set/test_audio44.wav', around_text=token_text.unsqueeze(0).to(device), verbose = True, 
                        language = 'ko', temperature=0.0) # compression_ratio_threshold = 0.1, logprob_threshold = -0.5, no_speech_threshold = 0.1)

[00:00.000 --> 00:30.000]  불매 불매 불매야 이기여등에 불매고 경상도 대불매 전라도야 자롱새 풀락 깍닥 불매야 느이 불매는 대불매 둘이 불매는 서불매 혼자 불매는 펜수컨 벼룻다 하면 혼미요 뚜둘겉다 하면 나시요 당겉다 하면 꽹이다 아홉다하면 황에다 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야 아홉에야


In [189]:
kk

{'text': ' 배야 이기 으 등에 불매고 경상도 대 불매 전라도 저렁새 훌락 깍닥 불매야 느이 불먼은 대불매 둘이 불먼은 서불매 혼자 불먼은 펜스캉 배렏다 하면 호미요 뚜둘겉다 하면 나시요 당겉다 하면 꽹이다',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 0.96,
   'text': ' 배야 이기 으 등에 불매고 경상도 대 불매 전라도 저렁새 훌락 깍닥 불매야 느이 불먼은 대불매 둘이 불먼은 서불매 혼자 불먼은 펜스캉 배렏다 하면 호미요 뚜둘겉다 하면 나시요 당겉다 하면 꽹이다',
   'tokens': [50412,
    14155,
    4248,
    2620,
    2401,
    37163,
    531,
    22205,
    1517,
    25746,
    28728,
    1313,
    9537,
    5979,
    3744,
    5971,
    25746,
    28728,
    19617,
    25574,
    13163,
    254,
    223,
    28319,
    11091,
    2457,
    20523,
    8394,
    235,
    36674,
    25746,
    28728,
    4248,
    34378,
    1129,
    25746,
    48150,
    2124,
    17691,
    114,
    2196,
    28728,
    21433,
    1129,
    25746,
    48150,
    2124,
    32558,
    114,
    2196,
    28728,
    36028,
    25746,
    48150,
    2124,
    10981,
    250,
    7785,
    45134,
    231,
    2391,
    2004,
    254,
    237,
    1235,
 

In [107]:
kk 

{'text': ' 이여이여 이여도 한양 라 한 마슬에 새첩한 놈은',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 11.41,
   'text': ' 이여이여 이여도 한양 라 한 마슬에 새첩한 놈은',
   'tokens': [50364,
    4329,
    245,
    105,
    21064,
    105,
    4329,
    245,
    1698,
    1838,
    4815,
    14973,
    22339,
    4815,
    6437,
    44532,
    1517,
    31184,
    4758,
    102,
    3049,
    10091,
    230,
    2124],
   'temperature': 0.0,
   'avg_logprob': -0.2246149444580078,
   'compression_ratio': 1.032258064516129,
   'no_speech_prob': 1.1295273161238129e-08}],
 'language': 'ko'}

In [38]:
import IPython.display as ipd 
import torchaudio 
#import Ipython.display as ipd 
wav, sr =torchaudio.load('/home/daewoong/userdata/test_set/test_audio0.wav')
ipd.Audio(wav.mean(0), rate=sr)


In [23]:
kk

{'text': '',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 1.0,
   'text': '',
   'tokens': [],
   'temperature': 1.0,
   'avg_logprob': -2.4186010360717773,
   'compression_ratio': 0.0,
   'no_speech_prob': 4.9349338837600953e-08,
   'words': []}],
 'language': 'ko'}

In [ ]:
import IPython.display as ipd 
import torchaudio 
#import Ipython.display as ipd 
wav, sr =torchaudio.load('/home/daewoong/userdata/danbi/final_tts_audio/lyric0-2.wav')
ipd.Audio(wav.mean(0), rate=sr)


In [17]:
pred = model(audio.to(device), around_text.to(device), tokens=)

In [13]:
model.logits(input_text.to(device), audio.to(device), around_text.to(device)).shape

torch.Size([2, 32, 51865])

In [11]:
pred.shape

NameError: name 'pred' is not defined

In [10]:
#audio, input_text, input_txt_attn, around_text, around_txt_attn 
model(audio.to(device), around_text.to(device), tokens=input_text.to(device))

UnboundLocalError: local variable 'audio_input' referenced before assignment

In [9]:
model.eval()
with torch.no_grad():
  model.transcribe('/home/daewoong/userdata/danbi/final_tts_audio/lyric0-2.wav')

TypeError: logits() missing 1 required positional argument: 'ref_features'